# Init

In [ ]:
# ----------- set params ----------- #

writefiles = True
render_maps = True

# buffer around input perimeter polygon
perimeter_buffer = 35_000 #meters

crs_global = 4326
crs_dem = 25832

In [ ]:
# ----------- init -----------
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
from datetime import date
import folium
from shapely.geometry import LineString, MultiLineString
import plotly.graph_objects as go
from collections import defaultdict
from datetime import datetime
from shapely import wkb
from shapely.geometry import Point
import rasterio
from rasterio.merge import merge
import numpy as np
from pathlib import Path
from shapely.geometry import box
import geopandas as gpd
import numpy as np


# custom skip magic
from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    pass  # Ignores execution

data_dir = Path.cwd().parent.parent / 'data' 


# ----------- attributes to retrieve -----------
ox.settings.useful_tags_way = [
    'highway', 'lanes', 
    'surface', 'lit', 'maxspeed', 'landuse', 'junction',
    'oneway', 'oneway:bicycle', 'bicycle', 
    'cycleway',  
    
    'cycleway:right', 'cycleway:left', 'cycleway:both',
    'cycleway:right:oneway', 'cycleway:left:oneway',

]

ox.settings.useful_tags_node = [
    'asl', 'bicycle_parking', 'cycleway'
]

# Fetch Network

In [ ]:

# ----------- load perimeter ----------#
polygon = gpd.read_file(data_dir / "city_perimeters/perimeter_copenhagen.geojson").to_crs(25832)
polygon = polygon.buffer(perimeter_buffer).to_crs(4326)

# clip area on  sweden
greater_copenhagen = gpd.read_file(data_dir / "city_perimeters/perimeter_greater_copenhagen.geojson").to_crs(4326)
polygon = gpd.clip(polygon, greater_copenhagen)


# Get OSM full network
cycle_graph = ox.graph_from_polygon(polygon.union_all(), network_type="all", simplify=False)

# Add bearings and edge lengths
cycle_graph = ox.add_edge_bearings(cycle_graph)
cycle_graph = ox.distance.add_edge_lengths(cycle_graph)

# Convert to GeoDataFrames
cycle_nodes, cycle_edges = ox.graph_to_gdfs(cycle_graph, nodes=True, edges=True)

# number rounding
cycle_edges["bearing"] = cycle_edges["bearing"].round()
cycle_edges["length"] = cycle_edges["length"].round(2)


# ----------- filter for highway keys -----------

cycle_edges = cycle_edges[(
        cycle_edges['highway'].isin([
            "primary", "primary_link",
            "secondary", "secondary_link", 
            "tertiary", "tertiary_link",
            "residential", "living_street", "service", 
            "unclassified", "track", 
            "road", 
            "cycleway",
            "pedestrian", "footway", "path", "steps"])  |
    cycle_edges['cycleway'].isin(["lane", "shared_lane", "share_busway", "track" ]) |
    cycle_edges['cycleway:right'].isin(["lane", "shared_lane", "share_busway", "track" ]) |
    cycle_edges['cycleway:both'].isin(["lane", "shared_lane", "share_busway", "track" ]) |
    cycle_edges['cycleway:left'].isin(["lane", "shared_lane", "share_busway", "track" ])
)]


# Reset Indices
cycle_edges.reset_index(inplace = True, drop = False)
cycle_nodes.reset_index(inplace = True, drop = False)

# drop OSMID because it's of no practical use
cycle_edges = cycle_edges.drop(columns = 'osmid')


# ----------- replace nan strings with nan -----------
cycle_edges.replace("nan", np.nan, inplace=True)


## Clean Edges

In [ ]:
# -----------------------------------------------------------------------------
# ---- select links ----
# -----------------------------------------------------------------------------

# init temp column to indicate manually reversed links
cycle_edges['reversed_manually'] = False

def flip_link(row):
    reversed_row = row.copy()
    # Swap u and v
    reversed_row['u'], reversed_row['v'] = row['v'], row['u']
    # Invert bearing
    reversed_row['bearing'] = (row['bearing'] + 180) % 360
    # Invert reversed boolean
    reversed_row['reversed'] = not row['reversed']
    # swap left / right cycleway
    reversed_row['cycleway:left'], reversed_row['cycleway:right'] = reversed_row['cycleway:right'], reversed_row['cycleway:left']
    # Reverse geometry
    reversed_row['geometry'] = LineString(row['geometry'].coords[::-1])
    # Indicate newly created twin
    reversed_row['reversed_manually'] = True
    return reversed_row


# ----------- function to select links conditionally -----------


def select_links(link):
    out = []
    # for oneway links with bicycle counterflow, insert reverse link (will be filtered for duplicates later on)
    if link['oneway']: #and link['oneway:bicycle'] == 'no'
        reverse_link = flip_link(link)
        out.append(reverse_link)
    out.append(link)       
    return out

# run function
selected_rows = []
for _, row in cycle_edges.iterrows():
    result = select_links(row)
    if result:
        selected_rows.extend(result)

cycle_edges = gpd.GeoDataFrame(selected_rows, geometry='geometry', crs=crs_global)

# ----------- determine most relevant cycleway type -----------
# source hierarchy is 
# 1 cycleway - 48k entries
# 2 cycleway:both - 37k entries
# 3 cycleway:right - 14k entries 
# - bicycle

def assign_main_cycleway_type(link):
    # 1 highway = cycleway
    if link['highway'] == "cycleway":
            main_cycleway_type = 'cycleway'
    # 2 cycleway key 
    elif pd.notna(link['cycleway']) and link['cycleway'] not in ["no", "nan"]:
        main_cycleway_type = str(link['cycleway']).replace("opposite_", "")
    # 3 cycleway:both key 
    elif pd.notna(link['cycleway:both']) and link['cycleway:both'] not in ["no", "nan"]:
        main_cycleway_type = str(link['cycleway:both']).replace("opposite_", "")
    # 4 cycleway:right key
    elif pd.notna(link['cycleway:right']) and link['cycleway:right'] not in ["no", "nan"]:
        main_cycleway_type = str(link['cycleway:right']).replace("opposite_", "")
    # X cycling on pedestrian ways ( but not stairs / steps )
    elif link['highway'] in ['pedestrian', 'footway', 'path']:
            # Xa permitted implicitly. even if "bicycle = no" is set, this is implemented
            # inconsistently and not followed by observed cyclist tracks
            if link['bicycle'] in ['no']:
                main_cycleway_type = "shared_pedestrian_disallowed"
            else:
                main_cycleway_type = "shared_pedestrian"
    # X edge cases : no cycleway assigned but an explicitly permitted "bicycle" category
    elif pd.notna(link['bicycle']) and link['bicycle'] not in ['no', 'use_sidepath']:
        main_cycleway_type = link['bicycle']
        # double check, fallback value : "permitted"
        # if 'designated', per osm guidelines, there exists a cycleway attribute though
    # X cycling on shared lane with motor traffic
    elif link['highway'] not in ['pedestrian', 'path', 'footway', 'cycleway']:
        main_cycleway_type = "unguided"
    
    # final catcher
    else:
        main_cycleway_type = "undefined"

    # reassign after loop:
    if main_cycleway_type in ['designated', 'destination', 'dismount', 'permissive', 'private', 'yes']:
        main_cycleway_type = 'unguided_but_permitted_explicitly'
    if main_cycleway_type == "shared_lane":
        main_cycleway_type = "unguided"
    if main_cycleway_type in ['construction', 'crossing', 'link']:
        main_cycleway_type = 'lane'
    if main_cycleway_type in ['use_sidepath', 'separate','seperate', 'use_seperate']:
        main_cycleway_type = 'separate_track_indicated'
    if main_cycleway_type == 'share_busway':
        main_cycleway_type = 'shared_buslane'
    if main_cycleway_type == 'shared':
        main_cycleway_type = 'unguided'    
    if main_cycleway_type == 'opposite':
        main_cycleway_type = 'unguided'    
    if main_cycleway_type == 'shoulder':
        main_cycleway_type = 'unguided'     
    return main_cycleway_type

cycle_edges['cycleway_master'] = cycle_edges.apply(assign_main_cycleway_type, axis=1)

# ----------- turn into numbered item for possible comparisons -----------
# define hierarchy from most to least important
cycle_edges['cycleway_master'] = cycle_edges['cycleway_master'].replace({
    'cycleway': '1_cycleway',
    'track': '2a_track',
    'separate_track_indicated': '2b_separate_track_indicated',
    'lane': '3_lane',
    'shared_buslane': '4_shared_buslane',
    'unguided_but_permitted_explicitly': '5_unguided_but_permitted_explicitly',
    'shared_pedestrian': '6a_shared_pedestrian',
    'shared_pedestrian_disallowed': '6b_shared_pedestrian_disallowed',
    'unguided': '7_unguided'
})




In [ ]:
# assign nodes "is_junction" if 3 or more edges connected
# to differentiate between geometric nodes and true nodes

# combine u and v into one list of (node, neighbor)
pairs = pd.concat([
    cycle_edges[['u', 'v']],
    cycle_edges[['v', 'u']].rename(columns={'v': 'u', 'u': 'v'})
]).drop_duplicates()

# count how many unique neighbors each node has
deg = pairs.groupby('u')['v'].nunique()

cycle_nodes_assigned = cycle_nodes.copy()
cycle_nodes_assigned['is_junction'] = cycle_nodes_assigned['osmid'].map(deg).fillna(0).ge(3)

# keep only nodes referenced in edges
cycle_nodes_assigned = cycle_nodes_assigned[
    cycle_nodes_assigned['osmid'].isin(cycle_edges['u']) |
    cycle_nodes_assigned['osmid'].isin(cycle_edges['v'])
]

# ----------- enrich edges with END NODE data (attributes + is_junction) ----------- #
end_node_attrs = cycle_nodes_assigned.copy()
end_node_attrs = end_node_attrs.add_prefix('end_node_')
end_node_attrs = end_node_attrs.rename(columns={
    'end_node_osmid': 'v',
    'end_node_highway': 'end_node_amenity'
}).drop(columns=['end_node_x', 'end_node_y', 'end_node_geometry'])

cycle_edges = cycle_edges.merge(end_node_attrs, on='v', how='left')

# ----------- drop duplicates created in reversing links manually ----------- #
dupe_mask = cycle_edges.duplicated(subset=['u', 'v', 'key'], keep=False)
cycle_edges = cycle_edges[
    ~dupe_mask | (dupe_mask & (cycle_edges['reversed_manually'] != True))
]


In [ ]:
# ----------- cluster intersections -----------
from sklearn.cluster import DBSCAN

eps = 20

true_fork_nodes = cycle_nodes_assigned[cycle_nodes_assigned['is_junction']].to_crs(crs_dem)
#coords = [[cycle_nodes_pruned.geometry.x, cycle_nodes_pruned.geometry.y]]
coords = np.array(list(zip(true_fork_nodes.geometry.x, true_fork_nodes.geometry.y)))

# DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=2)
cluster_labels = dbscan.fit_predict(coords)

# Update cluster labels

true_fork_nodes['node_cluster'] = cluster_labels

intersection_cluster_key = true_fork_nodes[['osmid', 'node_cluster' ]]

# Group by 'cluster' and compute centroid of each group
true_fork_centroids = (
    true_fork_nodes
    .dissolve(by='node_cluster')  # merges by cluster
    .centroid
    .reset_index()
)

# Convert to GeoDataFrame
true_fork_centroids = gpd.GeoDataFrame(true_fork_centroids, geometry=0, crs=true_fork_nodes.crs)
true_fork_centroids = true_fork_centroids.rename(columns={0: 'geometry'}).set_geometry("geometry")


# ----------- get stop cluster information -----------
stop_clusters = pd.read_parquet(data_dir / 'hovding_processed/copenhagen_stop_clusters/copenhagen_stop_clusters_2501.parquet')
stop_clusters['geometry'] = [Point(xy) for xy in zip(stop_clusters['longitude'], stop_clusters['latitude'])]
stop_clusters = gpd.GeoDataFrame(stop_clusters, geometry='geometry', crs=crs_global).to_crs(crs_dem).drop(columns = ['cluster', 'latitude', 'longitude', 'intersection_id'])



# ----------- join stop clusters to intersection cluster centroid by nearest -----------
# Spatial join by nearest within 40 meters
intersections_enriched = gpd.sjoin_nearest(
    true_fork_centroids,
    stop_clusters,
    how='left',
    max_distance=40,
    distance_col='dist'
).drop(columns = ['index_right', 'geometry'])
    
intersections_enriched['dist'] = intersections_enriched['dist'].round(1)

intersections_enriched = intersections_enriched.merge(intersection_cluster_key, how = 'right', on = 'node_cluster')

cycle_nodes_assigned = cycle_nodes_assigned.merge(intersections_enriched, how = 'left', on= 'osmid')


cycle_edges = cycle_edges.merge(cycle_nodes_assigned[["osmid", "ratio_stopped", "mean_duration_stopped", "red_cycle_95pct"]], how = "left", 
                                          left_on = "v",
                                          right_on = "osmid",
                                          ).drop(columns = "osmid")

cycle_edges = cycle_edges.rename(columns = {'ratio_stopped' : 'end_node_ratio_stopped',
                                                    'mean_duration_stopped' : 'end_node_mean_duration_stopped',
                                                    'red_cycle_95pct' : 'end_node_red_phase_estimate'})



In [ ]:
# ----------- additions through corin : booleans -----------

cycle_edges['highway_is_pedestrian'] = ((cycle_edges['highway'] == 'pedestrian') | 
                                        (cycle_edges['highway'] == 'footway') | 
                                        (cycle_edges['highway'] == 'path')                                       
                                        )

cycle_edges['cycletrack_assumed_separate'] = (
    ((cycle_edges['cycleway:right'] == 'separate') & (~cycle_edges['reversed'])) |
    ((cycle_edges['cycleway:left'] == 'separate') & (cycle_edges['reversed']))
)


# ----------- no cycling expected ----------- #

cycle_edges['no_cycling_expected'] = (
    # reversed_manually is True AND
    (cycle_edges['reversed_manually'] == True) &
    (
        # either oneway:bicycle == 'yes'
        (cycle_edges['oneway:bicycle'] == 'yes') |
        # or oneway:bicycle is null AND oneway is not null AND oneway == True
        (
            cycle_edges['oneway:bicycle'].isna() &
            cycle_edges['oneway'].notna() &
            (cycle_edges['oneway'] == True)
        )
    )
) | (
    # OR highway is 'pedestrian' OR 'footway'
    cycle_edges['highway'].isin(['pedestrian', 'footway'])
)

# bool
cycle_edges['no_cycling_expected'] = cycle_edges['no_cycling_expected'].astype(bool)

# ----------- seperate cycle track expected ----------- #

cycle_edges['cycletrack_expected'] = (
    cycle_edges['cycletrack_assumed_separate'] |
    (
        cycle_edges['cycleway:right'].isna() &
        cycle_edges['bicycle'].notna() &
        (cycle_edges['bicycle'] == 'use_sidepath')
    )
)

# bool
cycle_edges['cycletrack_expected'] = cycle_edges['cycletrack_expected'].astype(bool)

# ----------- manually added links ----------- #
# load manual links
new_links = gpd.read_file("data/manually_added_links.gpkg")
# reproject to cycle_edges CRS
new_links = new_links.to_crs(cycle_edges.crs)
# append to cycle_edges
cycle_edges = pd.concat([cycle_edges, new_links], ignore_index=True)

In [ ]:
# -----------------------------------------------------------------------------
# ---- network ID looup ----
# -----------------------------------------------------------------------------
# Create unique network_id
cycle_edges["network_id"] = cycle_edges.index
cols = ["network_id"] + [col for col in cycle_edges.columns if col != "network_id"]
cycle_edges = cycle_edges[cols]


# ----------- lookup table -----------
lookup_table = cycle_edges[["network_id", "u", "v", "key"]]

## match DTM to points


In [ ]:
%%skip
wcs = WebCoverageService(
    
    'https://api.dataforsyningen.dk/dhm_wcs_DAF?service=WCS&request=GetCapabilities&token=769ae7f4d33cea021c8507aa4e7b7c16'
)

cycle_rd = cycle_nodes.to_crs('EPSG:25832')
minx, miny, maxx, maxy = cycle_rd.total_bounds
bbox = (minx, miny, maxx, maxy)

print(list(wcs.contents))

print([op.name for op in wcs.operations])

cov = wcs.contents['dhm_terraen']
print(cvg.supportedCRS)
print(cvg.boundingBoxWGS84)
print(cvg.supportedFormats)

# Request the DSM data from the WCS
response = wcs.getCoverage(identifier='dhm_terraen', 
                           bbox=bbox, 
                           format='GEOTIFF',
                           crs='urn:ogc:def:crs:EPSG::25832') #, resx=0.5, resy=0.5)


In [ ]:
%%skip
from owslib.wcs import WebCoverageService
import math, os, rasterio

# -------------------------------------------------------------------------
WCS = WebCoverageService(
    'https://api.dataforsyningen.dk/dhm_wcs_DAF'
    '?service=WCS&request=GetCapabilities'
    '&token=769ae7f4d33cea021c8507aa4e7b7c16',
    version='1.0.0'
)

CRS_CODE = "EPSG:25832"                 
CRS_URN  = "urn:ogc:def:crs:EPSG::25832"
PIXEL    = 0.4                          # meters
TILE     = 100
cycle_utm = cycle_nodes.to_crs(CRS_CODE)
minx, miny, maxx, maxy = map(float, cycle_utm.total_bounds)

print(f"requesting {width} × {height} pixels")
cycle_utm = cycle_nodes.to_crs(CRS_CODE)
minx, miny, maxx, maxy = map(float, cycle_utm.total_bounds)


for x0 in np.arange(minx, maxx, TILE):
    for y0 in np.arange(miny, maxy, TILE):
        bbox  = (x0, y0, x0 + TILE, y0 + TILE)
        width = math.ceil(TILE / PIXEL)
        height = width
        resp = WCS.getCoverage(
            identifier='dhm_terraen',
            bbox=bbox,
            crs=CRS_URN,
            format='GTiff',
            width=width,
            height=height
        )
        print(resp.read())
        data = resp.read()
        fname = f"data/dhm_terraen_{int(x0)}_{int(y0)}.tif"
        with open(fname, "wb") as f:
            f.write(data)

In [ ]:
WCS = WebCoverageService(
    "https://api.dataforsyningen.dk/dhm_wcs_DAF"
    "?service=WCS&request=GetCapabilities"
    "&token=769ae7f4d33cea021c8507aa4e7b7c16",
    version="1.0.0"
)

CRS_CODE = "EPSG:25832"
CRS_URN  = "urn:ogc:def:crs:EPSG::25832"
PIXEL    = 0.4               # m
TILE     = 500               # m
BATCH    = 20
RESUME_FROM_FILE = False
# --------------
# -------------------------------------------------------------------------

save_pq  = Path("temp/cycle_nodes_dtm.parquet")
#out_dir.mkdir(exist_ok=True, parents=True)

if save_pq.exists() and RESUME_FROM_FILE:
    dtm = gpd.read_parquet(save_pq)
    #ele_done = dtm[dtm["altitude_dtm"].notna()]
    #ele_todo = dtm[dtm["altitude_dtm"].isna()]

else:
    dtm = cycle_nodes.to_crs(CRS_CODE)[['osmid', 'geometry']]
    dtm["altitude_dtm"] = np.nan
    #dtm = gpd.GeoDataFrame()

def grid(bounds, size):
    minx, miny, maxx, maxy = bounds
    for x in np.arange(minx, maxx, size):
        for y in np.arange(miny, maxy, size):
            yield (x, y, x + size, y + size)


counter = 0
for bx in grid(dtm[dtm['altitude_dtm'].isna()].total_bounds, TILE):
    idx = dtm.altitude_dtm.isna() & dtm.geometry.within(box(*bx))
    if not idx.any():
        continue

    w = h = math.ceil(TILE / PIXEL)
    resp = WCS.getCoverage(
        identifier="dhm_terraen",
        bbox=bx,
        crs=CRS_URN,
        format="GTiff",
        width=w,
        height=h
    )
    tif = resp.read()

    with MemoryFile(tif) as mem, mem.open() as src:
        coords = [(pt.x, pt.y) for pt in dtm.loc[idx, "geometry"]]
        vals   = [v[0] for v in src.sample(coords)]
        nd     = src.nodata
    dtm.loc[idx, "altitude_dtm"] = [np.nan if v == nd else round(v, 3) for v in vals]
    dtm['altitude_dtm'] = dtm['altitude_dtm'].round(3)
    counter += 1
    if counter % BATCH == 0:
        dtm.to_parquet(save_pq, index=False)

dtm.to_parquet(save_pq, index=False)




# ----------- merge to edges -----------

# Merge for ele_orig
cycle_edges = cycle_edges.merge(
    dtm[['osmid', 'altitude_dtm']],
    how='left',
    left_on='u',
    right_on='osmid'
).rename(columns={'altitude_dtm': 'ele_orig'}).drop(columns='osmid')

# Merge for ele_dest
cycle_edges = cycle_edges.merge(
    dtm[['osmid', 'altitude_dtm']],
    how='left',
    left_on='v',
    right_on='osmid'
).rename(columns={'altitude_dtm': 'ele_dest'}).drop(columns='osmid')

cycle_edges['gradient'] = (cycle_edges['ele_dest'] - cycle_edges['ele_orig']) / cycle_edges['length']


cycle_edges['ele_orig'] = cycle_edges['ele_orig'].round(2)
cycle_edges['ele_dest'] = cycle_edges['ele_dest'].round(2)
cycle_edges['gradient'] = cycle_edges['gradient'].round(5)


In [ ]:
# ----------- remove duplicate links -----------
cycle_edges = cycle_edges.drop_duplicates(subset=['u', 'v', 'key'], keep='first')


### Cleanup data types

In [ ]:
cycle_edges = cycle_edges.astype({
    'u': 'Int64',
    'v': 'Int64',
    'key': 'Int8',
    'highway': 'string',
    'lanes': 'Int8',
    'surface': 'string',
    'lit': 'string',
    'maxspeed': 'Int16',
    'oneway': 'bool',
    'reversed': 'bool',
    'reversed_manually': 'bool',
    'cycleway': 'string',
    'bicycle': 'string',
    'junction': 'string',
    'oneway:bicycle': 'string',
    'cycleway:both': 'string',
    'cycleway:left': 'string',
    'cycleway:right': 'string',
    'cycleway:left:oneway': 'string',
    'cycleway_master': 'string',
    'end_node_street_count': 'Int64',
    'end_node_is_junction': 'bool',
    'end_node_ratio_stopped': 'float32',
    })

cycle_edges['length'] = cycle_edges['length'].round(2).astype('float32')
cycle_edges['bearing'] = cycle_edges['bearing'].round().astype('Int16')
cycle_edges['end_node_red_phase_estimate'] = cycle_edges['end_node_red_phase_estimate'].round().astype('Int16')
cycle_edges['end_node_ratio_stopped'] = cycle_edges['end_node_ratio_stopped'].round(4).astype('float32')
cycle_edges['end_node_mean_duration_stopped'] = cycle_edges['end_node_mean_duration_stopped'].round(2).astype('float32')


# Write

In [ ]:
gpd.options.io_engine = "pyogrio"

# -----------------------------------------------------------------------------
# ---- write files ----
# -----------------------------------------------------------------------------

if writefiles:
        
    #today_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    today_str = datetime.now().strftime("%Y-%m-%d")
    
    cycle_edges_out_extended = cycle_edges.to_crs(32632)
    cycle_edges_out_short = cycle_edges_out_extended[["network_id", "highway", "cycleway_master", "geometry"]]

    # ----------- parquet ----------- # 
    #cycle_edges_out_extended.to_parquet(f"out/osm_nodes_MobiJoule_{today_str}.parquet")
    #cycle_edges_out_short.to_parquet(f"out/osm_edges_MobiJoule_{today_str}_short.parquet")

    # ----------- gpkg ----------- # 
    cycle_edges_out_extended.to_file(f"output/osm_edges_MobiJoule_{today_str}.gpkg", layer='cycle_edges', driver="GPKG")
    #cycle_edges_out_short.to_file(f"output/osm_edges_MobiJoule_{today_str}_short.gpkg", layer='cycle_edges', driver="GPKG")
    
    # ----------- lookup table ----------- #  
    lookup_table.to_csv(f"output/osm_edges_MobiJoule_{today_str}_id_lookup.csv", index=False)
        
        
    # csv
    #cycle_edges_csv = cycle_edges.drop(columns = "geometry")
    #cycle_edges_csv.to_csv(f"output/osm_edges_MobiJoule_{today_str}.csv", index=False)
